In [3]:
import numpy as np
import pandas as pd
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# load in data
df2021 = pd.read_csv('2021VAERSSYMPTOMS.csv')
df2022 = pd.read_csv('2022VAERSSYMPTOMS.csv')
dfsymptoms = pd.concat([df2021, df2022], axis=0)

In [4]:
# combine duplicate rows in SYMPTOMS file
dfsymptoms_merged = dfsymptoms.groupby(['VAERS_ID'])[['SYMPTOM1','SYMPTOMVERSION1', 'SYMPTOM2', 'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3', 'SYMPTOM4', 'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5']].agg(set)

In [51]:
# merge VAX and SYMPTOMS files based on VAX file
dfvax2021 = pd.read_csv('2021VAERSVAX.csv')
dfvax2022 = pd.read_csv('2022VAERSVAX.csv')
dfvax = pd.concat([dfvax2021, dfvax2022], axis=0)
dfmerged = pd.merge(dfvax, dfsymptoms, how="outer", on="VAERS_ID")
dfmerged.head(1)

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,910642,COVID19,PFIZER\BIONTECH,EJ1685,1,NaN,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),Dyspnoea,25.0,Heart rate increased,25.0,SARS-CoV-2 test negative,25.0,NaN,NaN,NaN,NaN


In [52]:
# extracting SYMPTOM columns
dfmerged_symp = dfmerged[["VAERS_ID","VAX_MANU","SYMPTOM1","SYMPTOM2","SYMPTOM3","SYMPTOM4","SYMPTOM5"]]

In [33]:
dfmerged["ALLSYMPTOMS"] = dfmerged["SYMPTOM1"].str.cat...
    (dfmerged_symp[["SYMPTOM2", "SYMPTOM3", "SYMPTOM4", "SYMPTOM5"]].astype(str), sep=",")

In [43]:
# combining all symptoms into one column
dfmerged_symp = dfmerged[["VAERS_ID","VAX_MANU","ALLSYMPTOMS"]]

In [39]:
# group into the top five firms
dfpbt = dfmerged_symp[dfmerged_symp['VAX_MANU'] == 'PFIZER\BIONTECH'].reset_index(drop=True)
dfmoderna = dfmerged_symp[dfmerged_symp['VAX_MANU'] == 'MODERNA'].reset_index(drop=True)
dfjanssen = dfmerged_symp[dfmerged_symp['VAX_MANU'] == 'JANSSEN'].reset_index(drop=True)
dfgsk = dfmerged_symp[dfmerged_symp['VAX_MANU'] == 'GLAXOSMITHKLINE BIOLOGICALS'].reset_index(drop=True)
dfmerck = dfmerged_symp[dfmerged_symp['VAX_MANU'] == 'MERCK & CO. INC.'].reset_index(drop=True)

In [54]:
dfpbt.head()

,VAERS_ID,VAX_MANU,ALLSYMPTOMS
0,910642,PFIZER\BIONTECH,"Dyspnoea,Heart rate increased,SARS-CoV-2 test ..."
1,916602,PFIZER\BIONTECH,"Chest discomfort,Dysphagia,Pain in extremity,V..."
2,916617,PFIZER\BIONTECH,"Chills,Dizziness,Injection site pain,Myalgia,P..."
3,916619,PFIZER\BIONTECH,"Injection site pain,Menorrhagia,nan,nan,nan"
4,916623,PFIZER\BIONTECH,"Injection site pain,nan,nan,nan,nan"


In [40]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim
from gensim import corpora, models
from nltk.stem.wordnet import WordNetLemmatizer

In [67]:
def filterNoneType(firm):
    pbt = []
    for i in dfpbt.ALLSYMPTOMS: #filter out NoneType
        if type(i) == str:
            pbt.append(i)
    return pbt

In [68]:
filterNoneType(dfpbt)

['Dyspnoea,Heart rate increased,SARS-CoV-2 test negative,nan,nan',
 'Chest discomfort,Dysphagia,Pain in extremity,Visual impairment,nan',
 'Chills,Dizziness,Injection site pain,Myalgia,Pyrexia',
 'Injection site pain,Menorrhagia,nan,nan,nan',
 'Injection site pain,nan,nan,nan,nan',
 'Chills,Dizziness,Nausea,Pain,Tachycardia',
 'Tremor,nan,nan,nan,nan',
 'Chills,Diarrhoea,Fatigue,Feeling abnormal,Headache',
 'Hot flush,Myalgia,Nausea,Tension headache,nan',
 'Hypoaesthesia,Pain in extremity,Peripheral swelling,Ultrasound scan,nan',
 'Chills,Headache,Hyperhidrosis,Pain,Pyrexia',
 'Pruritus,Rash,Urticaria,nan,nan',
 'Anosmia,Chills,Diarrhoea,Migraine,Pain',
 'Paraesthesia,Parosmia,Skin burning sensation,Taste disorder,nan',
 'Feeling cold,Hypoaesthesia,Paraesthesia,nan,nan',
 'Chest X-ray normal,Computerised tomogram normal,Electrocardiogram normal,Facial paralysis,Headache',
 'Hypoaesthesia,nan,nan,nan,nan',
 'Arthralgia,Back pain,Dyspnoea,nan,nan',
 'Headache,Muscle spasms,Nausea,Pain,Py

In [71]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = nltk.corpus.stopwords.words('english')

# adding new stop words
new_stopwords = ['non', 'could', 'within']
en_stop = en_stop.extend(new_stopwords)

lemmatizer = WordNetLemmatizer()

# list for tokenized documents in loop
textspbt = []

# change type to string
dfpbt = dfpbt.astype(str)

# loop through document list
for i in dfpbt.ALLSYMPTOMS:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop] 
    
    # remove numbers from tokens    
    letter_tokens = [i for i in stopped_tokens if not any(filter(str.isdigit, i))]
    
    # stem tokens
    stemmed_tokens = [p_stemmer(i) for i in letter_tokens]

    # add tokens to list
    textspbt.append(stemmed_tokens)
    
# prints list vertically instead of horizontally
for i in textspbt:
    print(i)

TypeError: argument of type 'NoneType' is not iterable